In [287]:
# importing modules
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import pickle

In [26]:
# loading dataset 
user = pd.read_csv('Users.csv')
ratings = pd.read_csv('Ratings.csv')
books = pd.read_csv('Books.csv')

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [27]:
# data pre-processing on users dataset
user.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [28]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [29]:
user.shape

(278858, 3)

In [30]:
user.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [31]:
user.dropna(inplace=True)

In [32]:
user.isnull().sum()

User-ID     0
Location    0
Age         0
dtype: int64

In [33]:
user.duplicated().sum()

0

In [65]:
# data pre-processing on ratings dataset
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [66]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [67]:
books_data = books[['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher']]

In [68]:
books_data.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [69]:
books_data.shape

(271360, 5)

In [70]:
books_data.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
dtype: int64

In [71]:
# data pre-processing on ratings dataset
ratings.head()

,User-ID,ISBN,Book-Rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [72]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526356 entries, 1456 to 1147616
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   User-ID      526356 non-null  int64 
 1   ISBN         526356 non-null  object
 2   Book-Rating  526356 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 16.1+ MB


In [73]:
ratings.shape

(526356, 3)

In [74]:
# ratings by unique users 
ratings['User-ID'].value_counts().shape

(899,)

In [75]:
ratings['User-ID'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
59727       201
188951      201
268622      201
9856        201
155916      201
Name: User-ID, Length: 899, dtype: int64

In [76]:
# extract the user-ids who have given more than 200 ratings and when we will have user-ids
# we will extract the ratings of only these user-id from the rating dataframe.

In [46]:
x = ratings['User-ID'].value_counts() > 200
y = x[x].index  #user_ids
print(y.shape)

(899,)


In [52]:
# y contain the user-id of those users which give review on more than 200 books (899 users give rating on more than 200 books)
y

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
             88793, 274808,  44296,  28634,  73681,  59727, 188951, 268622,
              9856, 155916],
           dtype='int64', length=899)

In [54]:
ratings = ratings[ratings['User-ID'].isin(y)]
# this will return only those rows in which user-id in present in y

In [55]:
ratings.shape

(526356, 3)

In [56]:
ratings.head()

,User-ID,ISBN,Book-Rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [63]:
# merge ratings with books on basis of ISBN so that we will get the rating of each user on each book id and 
# the user who has not rated that book id the value will be zero.

In [78]:
rating_with_books = ratings.merge(books_data, on='ISBN')
rating_with_books

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
...,...,...,...,...,...,...,...
487666,275970,1892145022,0,Here Is New York,E. B. White,1999,Little Bookroom
487667,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA)
487668,275970,3411086211,10,Die Biene.,Sybil GrÃ?Â¤fin SchÃ?Â¶nfeldt,1993,"Bibliographisches Institut, Mannheim"
487669,275970,3829021860,0,The Penis Book,Joseph Cohen,1999,Konemann


In [143]:
rating_with_books.shape  
# the ratings decreases from 526356 to 487671 because the books which do not have data are removed.

(487671, 7)

In [144]:
# grouping the book-title with the no. of times it is rated by any user
number_rating = rating_with_books.groupby('Book-Title')['Book-Rating'].count().reset_index()

In [145]:
number_rating.rename(columns= {'Book-Rating':'number_of_ratings'}, inplace=True)

In [146]:
number_rating.head()

,Book-Title,number_of_ratings
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [147]:
number_rating.shape

(160269, 2)

In [148]:
number_rating[number_rating['Book-Title']=="Die Biene."]

,Book-Title,number_of_ratings
33794,Die Biene.,1


In [149]:
final_rating = rating_with_books.merge(number_rating, on='Book-Title')

In [150]:
final_rating

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,number_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
...,...,...,...,...,...,...,...,...
487666,275970,1892145022,0,Here Is New York,E. B. White,1999,Little Bookroom,1
487667,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA),1
487668,275970,3411086211,10,Die Biene.,Sybil GrÃ?Â¤fin SchÃ?Â¶nfeldt,1993,"Bibliographisches Institut, Mannheim",1
487669,275970,3829021860,0,The Penis Book,Joseph Cohen,1999,Konemann,1


In [151]:
final_rating.shape

(487671, 8)

In [152]:
# selecting the books which have >= 50 ratings on them
final_rating = final_rating[final_rating['number_of_ratings'] >= 50]

In [153]:
final_rating

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,number_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
...,...,...,...,...,...,...,...,...
236701,255489,0553579983,7,And Then You Die,Iris Johansen,1998,Bantam,50
236702,256407,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,50
236703,257204,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,50
236704,261829,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,50


In [154]:
# drop duplicate values because if the same user has rated the same book multiple times so it will create a problem.
final_rating.drop_duplicates(['User-ID','Book-Title'], inplace=True)

C:\Users\user\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [155]:
# we have a dataset with those users who has rated more than 200 books and 
# books that received more than 50 ratings
final_rating

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,number_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
...,...,...,...,...,...,...,...,...
236701,255489,0553579983,7,And Then You Die,Iris Johansen,1998,Bantam,50
236702,256407,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,50
236703,257204,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,50
236704,261829,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,50


In [156]:
book_pivot = final_rating.pivot_table(columns='User-ID', index='Book-Title', values='Book-Rating')
book_pivot.fillna(0, inplace=True)

In [157]:
book_pivot

User-ID,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
# some users have removed out because their ratings were on those books
# which do not receive more than 50 ratings so they are moved out of the picture.   (899 -> 888)
book_pivot.columns.value_counts().sum()

888

In [159]:
# we use nearest neighbors algorithm which is the same as K nearest which is used for clustering based on euclidian distance.
# But here in the pivot table, we have lots of zero values and on clustering,
# this computing power will increase to calculate the distance of zero values
# so we will convert the pivot table to the sparse matrix and then feed it to the model.

In [160]:
book_sparse = csr_matrix(book_pivot)

In [161]:
type(book_sparse)

scipy.sparse.csr.csr_matrix

In [162]:
print(book_sparse.shape)

(742, 888)


In [163]:
# format of book_sparse >>> (row, column) value
print(book_sparse)

  (0, 0)	9.0
  (0, 16)	8.0
  (0, 37)	9.0
  (0, 42)	8.0
  (0, 59)	7.0
  (0, 135)	9.0
  (0, 156)	10.0
  (0, 170)	9.0
  (0, 187)	8.0
  (0, 220)	10.0
  (0, 221)	10.0
  (0, 245)	10.0
  (0, 277)	10.0
  (0, 359)	10.0
  (0, 370)	9.0
  (0, 389)	10.0
  (0, 477)	9.0
  (0, 491)	7.0
  (0, 507)	9.0
  (0, 554)	10.0
  (0, 604)	9.0
  (0, 611)	8.0
  (0, 695)	9.0
  (0, 734)	9.0
  (0, 762)	8.0
  :	:
  (740, 484)	8.0
  (740, 681)	9.0
  (741, 16)	8.0
  (741, 26)	7.0
  (741, 86)	10.0
  (741, 111)	5.0
  (741, 150)	6.0
  (741, 233)	7.0
  (741, 290)	8.0
  (741, 303)	9.0
  (741, 322)	10.0
  (741, 367)	10.0
  (741, 384)	9.0
  (741, 446)	7.0
  (741, 450)	9.0
  (741, 485)	7.0
  (741, 498)	10.0
  (741, 505)	8.0
  (741, 544)	10.0
  (741, 592)	8.0
  (741, 700)	10.0
  (741, 713)	8.0
  (741, 745)	10.0
  (741, 830)	9.0
  (741, 880)	8.0


In [142]:
# we need to specify an algorithm which is brute means find the distance of every point to every other point.

In [166]:
model = NearestNeighbors(algorithm = 'brute')

In [167]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [202]:
book_pivot.index[237]

'Harry Potter and the Chamber of Secrets (Book 2)'

In [196]:
# parameters of kneighbors(X, n_neighbors)
# x is the input that is one row of book_pivot (the book whose similar books we have to find)

distances, suggestions = model.kneighbors(book_pivot.iloc[237, :].values.reshape(1,-1), n_neighbors=6)
# this function return two arrays, one store distances and other store ids of suggestions

In [197]:
distances.shape

(1, 6)

In [198]:
distances

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [199]:
suggestions.shape

(1, 6)

In [200]:
suggestions

array([[237, 240, 238, 241, 184, 536]], dtype=int64)

In [255]:
for i in range(len(suggestions)):
     print(book_pivot.index[suggestions[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='Book-Title')


In [206]:
# finding index of book on the basis of its title
np.where(book_pivot.index == 'Animal Farm')[0][0]  

54

In [277]:
# creating a function which will take book name and gives suggestions
def recommendation(name):
    book_id = np.where(book_pivot.index == name)[0][0]
    distances, suggestions = model.kneighbors(book_pivot.iloc[book_id, :].values.reshape(1,-1), n_neighbors=6)
    
    for i in range(len(suggestions)):
        if i == 0:
            print("The suggestions are:")
        if not i:
            var = book_pivot.index[suggestions[i]]
            print(var.to_list())

In [278]:
recommendation('Harry Potter and the Chamber of Secrets (Book 2)')

The suggestions are:
['Harry Potter and the Chamber of Secrets (Book 2)', 'Harry Potter and the Prisoner of Azkaban (Book 3)', 'Harry Potter and the Goblet of Fire (Book 4)', "Harry Potter and the Sorcerer's Stone (Book 1)", 'Exclusive', 'The Cradle Will Fall']


In [284]:
# creating a new dataset to show the names of books on are web application
books_name = final_rating['Book-Title'].unique()

In [285]:
books_name

array(['Politically Correct Bedtime Stories: Modern Tales for Our Life and Times',
       'The Poisonwood Bible: A Novel', 'Bel Canto: A Novel',
       'One for the Money (Stephanie Plum Novels (Paperback))',
       'The Secret Garden', 'The Tao of Pooh', 'Girl in Hyacinth Blue',
       'Chocolat', 'The Secret Life of Bees',
       'Three To Get Deadly : A Stephanie Plum Novel (A Stephanie Plum Novel)',
       "Full Tilt (Janet Evanovich's Full Series)", 'Lucky : A Memoir',
       'The Dogs of Babel (Today Show Book Club #12)',
       "White Oleander : A Novel (Oprah's Book Club)",
       'White Oleander : A Novel', 'The Jester',
       'The Lovely Bones: A Novel', 'Me Talk Pretty One Day', 'Naked',
       'The Hobbit : The Enchanting Prelude to The Lord of the Rings',
       'A Prayer for Owen Meany', 'Silent Witness', 'No Safe Place',
       'The Murder Book', 'Middlesex: A Novel', 'Postmortem',
       'All That Remains (Kay Scarpetta Mysteries (Paperback))',
       'The Rainmaker',


In [288]:
pickle.dump(books_name,open('booksname.pkl','wb'))

In [289]:
pickle.dump(book_pivot,open('bookpivot.pkl','wb'))

In [290]:
pickle.dump(model, open('final_model.pkl', 'wb'))

In [332]:
books_info = books[['Book-Title','Image-URL-M']]

In [336]:
# finding image-url of book on the basis of its title
books_info[books_info['Book-Title']=='Jacob Have I Loved'].values[0][1]

'http://images.amazon.com/images/P/0380564998.01.MZZZZZZZ.jpg'

In [334]:
books_info

,Book-Title,Image-URL-M
0,Classical Mythology,http://images.amazon.com/images/P/0195153448.0...
1,Clara Callan,http://images.amazon.com/images/P/0002005018.0...
2,Decision in Normandy,http://images.amazon.com/images/P/0060973129.0...
3,Flu: The Story of the Great Influenza Pandemic...,http://images.amazon.com/images/P/0374157065.0...
4,The Mummies of Urumchi,http://images.amazon.com/images/P/0393045218.0...
...,...,...
271355,There's a Bat in Bunk Five,http://images.amazon.com/images/P/0440400988.0...
271356,From One to One Hundred,http://images.amazon.com/images/P/0525447644.0...
271357,Lily Dale : The True Story of the Town that Ta...,http://images.amazon.com/images/P/006008667X.0...
271358,Republic (World's Classics),http://images.amazon.com/images/P/0192126040.0...


In [335]:
pickle.dump(books_info,open('booksinfo.pkl','wb'))